# Demand Flexibility Model Comparison

This notebook compares two approaches to modeling demand-side flexibility in PyPSA-GB:

1. **Detailed Model**: Uses Store/Link components for explicit SOC tracking
   - EV batteries: Store with time-varying e_min_pu, Links for charge/discharge
   - Hot water tanks: Store with standing losses, Link with COP efficiency
   - Creates many variables and time-coupled constraints

2. **Simplified Model**: Uses load-shifting approach without explicit storage
   - Generator (negative) for upward flex (demand reduction)
   - Load for downward flex (demand increase/recovery)
   - Energy conservation enforced via daily constraints
   - Fewer variables, faster solve times

## Comparison Metrics
- Solve time
- Number of variables/constraints
- System cost
- Flexibility utilization patterns
- Peak demand reduction

**Note**: This notebook runs a 1-day (24h) simulation for speed.

In [1]:
# Import required libraries
import os
import sys
import time
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import copy

# Configure matplotlib
plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')
%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print('Libraries imported successfully')
print(f'Project root: {project_root}')

Libraries imported successfully
Project root: c:\Users\alyden\OneDrive - University of Edinburgh\Python\PyPSA-GB v0.0.1


## 1. Load Base Network

Load a solved network without flexibility as the baseline, then we'll add flexibility in two different ways.

In [ ]:
# Find available network files
network_dir = project_root / 'resources' / 'network'

# Look for a network with flexibility in the name (pre-solve)
available_networks = list(network_dir.glob('*_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc'))

if not available_networks:
    # Try solved networks
    available_networks = list(network_dir.glob('*_solved.nc'))

print('Available networks:')
for n in available_networks[:10]:
    print(f'  {n.name}')

# Select a network (prefer HT35 or similar future scenario)
network_path = None
for n in available_networks:
    if 'HT35' in n.name or 'flex' in n.name:
        network_path = n
        break

if network_path is None and available_networks:
    network_path = available_networks[0]

if network_path:
    print(f'\nSelected network: {network_path.name}')
else:
    print('\nNo suitable network found. Please ensure you have run the PyPSA-GB workflow first.')
    raise FileNotFoundError('No network files found')

network_path = 'HT35_clustered_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc'

Available networks:
  EE50_clustered_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc
  Historical_2015_reduced_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc
  Historical_2023_etys_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc
  HT35_clustered_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc
  HT35_flex_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc
  HT35_year_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc

Selected network: HT35_clustered_network_demand_renewables_thermal_generators_storage_hydrogen_interconnectors.nc


In [3]:
# Load the network
n_base = pypsa.Network(str(network_path))

print(f'Network loaded:')
print(f'  Buses: {len(n_base.buses)}')
print(f'  Generators: {len(n_base.generators)}')
print(f'  Loads: {len(n_base.loads)}')
print(f'  Links: {len(n_base.links)}')
print(f'  Stores: {len(n_base.stores)}')
print(f'  Snapshots: {len(n_base.snapshots)}')
print(f'  Period: {n_base.snapshots[0]} to {n_base.snapshots[-1]}')

INFO:pypsa.network.io:Imported network 'ETYS base + upgrades (2035)' has buses, carriers, generators, lines, links, loads, storage_units, stores, transformers


Network loaded:
  Buses: 2508
  Generators: 4985
  Loads: 313
  Links: 282
  Stores: 1
  Snapshots: 8760
  Period: 2035-01-01 00:00:00 to 2035-12-31 23:00:00


In [4]:
# Reduce to 1 day (24 hours) for faster comparison
ONE_DAY_HOURS = 24

# Select first 24 snapshots
if len(n_base.snapshots) > ONE_DAY_HOURS:
    snapshots_day = n_base.snapshots[:ONE_DAY_HOURS]
    print(f'Reduced to 1 day: {snapshots_day[0]} to {snapshots_day[-1]}')
else:
    snapshots_day = n_base.snapshots
    print(f'Using all {len(snapshots_day)} snapshots')

# Create a copy with reduced snapshots
def reduce_network_to_snapshots(network, snapshots):
    """Create a new network with only the specified snapshots."""
    n = network.copy()
    n.set_snapshots(snapshots)
    
    # Slice all time-varying data
    for component in ['generators', 'loads', 'storage_units', 'stores', 'links', 'lines']:
        comp_t = getattr(n, f'{component}_t')
        for attr in comp_t.keys():
            df = comp_t[attr]
            if len(df) > 0:
                # Filter to snapshots that exist in the dataframe
                valid_snapshots = [s for s in snapshots if s in df.index]
                if valid_snapshots:
                    comp_t[attr] = df.loc[valid_snapshots]
    
    return n

n_day = reduce_network_to_snapshots(n_base, snapshots_day)
print(f'Day network: {len(n_day.snapshots)} snapshots')

Reduced to 1 day: 2035-01-01 00:00:00 to 2035-01-01 23:00:00
Day network: 24 snapshots


## 2. Define Test Parameters

Set up parameters for the flexibility models.

In [5]:
# Flexibility parameters
FLEX_PARAMS = {
    # EV parameters
    'ev_participation': 0.10,  # 10% of EV fleet participates in flexibility
    'ev_battery_capacity_kwh': 60.0,  # Average battery size
    'ev_charger_power_kw': 7.0,  # Home charger power
    'ev_charge_efficiency': 0.90,
    'ev_discharge_efficiency': 0.90,  # V2G
    'ev_min_soc': 0.20,
    'ev_demand_fraction': 0.08,  # EV demand as fraction of total
    
    # Heat pump parameters
    'hp_participation': 0.30,  # 30% participate in flexibility
    'hp_tank_volume_liters': 200,
    'hp_tank_temp_range': [50, 65],  # degrees C
    'hp_standing_loss': 0.01,  # per hour
    'hp_cop': 3.0,
    'hp_demand_fraction': 0.15,  # HP demand as fraction of total
    
    # Simplified model parameters
    'shift_window_hours': 4,  # Maximum hours demand can be shifted
    'rebound_factor': 1.0,  # Energy payback ratio (1.0 = energy neutral)
}

print('Flexibility Parameters:')
for k, v in FLEX_PARAMS.items():
    print(f'  {k}: {v}')

Flexibility Parameters:
  ev_participation: 0.1
  ev_battery_capacity_kwh: 60.0
  ev_charger_power_kw: 7.0
  ev_charge_efficiency: 0.9
  ev_discharge_efficiency: 0.9
  ev_min_soc: 0.2
  ev_demand_fraction: 0.08
  hp_participation: 0.3
  hp_tank_volume_liters: 200
  hp_tank_temp_range: [50, 65]
  hp_standing_loss: 0.01
  hp_cop: 3.0
  hp_demand_fraction: 0.15
  shift_window_hours: 4
  rebound_factor: 1.0


## 3. Model A: Detailed Storage Model

This model creates explicit Store and Link components for each flexible load.

In [6]:
def add_detailed_ev_flexibility(n, params):
    """
    Add detailed EV flexibility using Store/Link model.
    
    Components created per bus:
    - Bus: EV battery bus
    - Store: EV fleet battery (aggregated)
    - Link: EV charger (grid -> battery)
    - Link: V2G (battery -> grid) [optional]
    - Load: EV driving demand
    """
    # Get buses with demand
    load_buses = n.loads[n.loads.carrier != 'load_shedding'].bus.unique()
    
    # Calculate total EV demand
    total_demand = n.loads_t.p_set.sum().sum()  # MWh
    ev_demand_total = total_demand * params['ev_demand_fraction']
    
    # Add EV carrier if not exists
    if 'EV battery' not in n.carriers.index:
        n.add('Carrier', 'EV battery', co2_emissions=0.0)
    if 'EV charger' not in n.carriers.index:
        n.add('Carrier', 'EV charger', co2_emissions=0.0)
    if 'EV driving' not in n.carriers.index:
        n.add('Carrier', 'EV driving', co2_emissions=0.0)
    
    # Count of components added
    n_stores = 0
    n_links = 0
    n_loads = 0
    
    for bus in load_buses[:50]:  # Limit to 50 buses for performance
        battery_bus = f"{bus} EV battery"
        
        # Get demand at this bus
        bus_loads = n.loads[n.loads.bus == bus]
        if bus_loads.empty:
            continue
            
        # Get demand timeseries
        load_names = [l for l in bus_loads.index if l in n.loads_t.p_set.columns]
        if not load_names:
            continue
            
        bus_demand = n.loads_t.p_set[load_names].sum(axis=1)
        ev_demand_bus = bus_demand * params['ev_demand_fraction']
        
        # Skip if demand is too small
        if ev_demand_bus.max() < 0.1:  # < 100 kW
            continue
        
        # Estimate fleet size
        daily_demand_mwh = ev_demand_bus.sum() / (len(n.snapshots) / 24)
        n_vehicles = max(1, int(daily_demand_mwh * 1000 / 10))  # ~10 kWh/day per vehicle
        n_flex_vehicles = max(1, int(n_vehicles * params['ev_participation']))
        
        # Add battery bus
        if battery_bus not in n.buses.index:
            n.add('Bus', battery_bus, carrier='EV battery')
        
        # Add Store (EV battery)
        fleet_capacity_mwh = n_flex_vehicles * params['ev_battery_capacity_kwh'] / 1000
        n.add('Store',
              f"{bus} EV store",
              bus=battery_bus,
              carrier='EV battery',
              e_nom=fleet_capacity_mwh,
              e_cyclic=True,
              e_min_pu=params['ev_min_soc'])
        n_stores += 1
        
        # Add charger Link
        charger_power_mw = n_flex_vehicles * params['ev_charger_power_kw'] / 1000
        n.add('Link',
              f"{bus} EV charger",
              bus0=bus,
              bus1=battery_bus,
              carrier='EV charger',
              efficiency=params['ev_charge_efficiency'],
              p_nom=charger_power_mw)
        n_links += 1
        
        # Add driving demand Load
        n.add('Load',
              f"{bus} EV driving",
              bus=battery_bus,
              carrier='EV driving',
              p_set=ev_demand_bus)
        n_loads += 1
    
    print(f'Added detailed EV flexibility:')
    print(f'  Stores: {n_stores}')
    print(f'  Links: {n_links}')
    print(f'  Loads: {n_loads}')
    
    return n

In [7]:
def add_detailed_hp_flexibility(n, params):
    """
    Add detailed heat pump flexibility using Store/Link model.
    
    Components created per bus:
    - Bus: Heat bus
    - Store: Hot water tank
    - Link: Heat pump (electricity -> heat with COP)
    - Load: Hot water demand
    """
    load_buses = n.loads[n.loads.carrier != 'load_shedding'].bus.unique()
    
    # Add carriers
    if 'heat' not in n.carriers.index:
        n.add('Carrier', 'heat', co2_emissions=0.0)
    if 'hot water' not in n.carriers.index:
        n.add('Carrier', 'hot water', co2_emissions=0.0)
    if 'heat pump' not in n.carriers.index:
        n.add('Carrier', 'heat pump', co2_emissions=0.0)
    
    # Calculate tank capacity in kWh
    temp_diff = params['hp_tank_temp_range'][1] - params['hp_tank_temp_range'][0]
    tank_capacity_kwh = params['hp_tank_volume_liters'] * 4.186 * temp_diff / 3600
    
    n_stores = 0
    n_links = 0
    n_loads = 0
    
    for bus in load_buses[:50]:  # Limit for performance
        heat_bus = f"{bus} heat"
        
        bus_loads = n.loads[n.loads.bus == bus]
        if bus_loads.empty:
            continue
            
        load_names = [l for l in bus_loads.index if l in n.loads_t.p_set.columns]
        if not load_names:
            continue
            
        bus_demand = n.loads_t.p_set[load_names].sum(axis=1)
        hp_demand_bus = bus_demand * params['hp_demand_fraction']
        
        if hp_demand_bus.max() < 0.1:
            continue
        
        # Estimate number of dwellings
        peak_mw = hp_demand_bus.max()
        n_dwellings = max(1, int(peak_mw * 1000 / 1.14))  # ~1.14 kW per dwelling
        n_flex_dwellings = max(1, int(n_dwellings * params['hp_participation']))
        
        # Add heat bus
        if heat_bus not in n.buses.index:
            n.add('Bus', heat_bus, carrier='heat')
        
        # Add hot water tank Store
        total_tank_mwh = n_flex_dwellings * tank_capacity_kwh / 1000
        n.add('Store',
              f"{bus} hot water tank",
              bus=heat_bus,
              carrier='hot water',
              e_nom=total_tank_mwh,
              e_cyclic=True,
              standing_loss=params['hp_standing_loss'])
        n_stores += 1
        
        # Add heat pump Link (with COP as efficiency)
        heater_power_mw = n_flex_dwellings * 3.0 / 1000  # 3 kW per dwelling
        n.add('Link',
              f"{bus} heat pump",
              bus0=bus,
              bus1=heat_bus,
              carrier='heat pump',
              efficiency=params['hp_cop'],
              p_nom=heater_power_mw)
        n_links += 1
        
        # Add hot water demand Load
        n.add('Load',
              f"{bus} hot water demand",
              bus=heat_bus,
              carrier='hot water',
              p_set=hp_demand_bus)
        n_loads += 1
    
    print(f'Added detailed HP flexibility:')
    print(f'  Stores: {n_stores}')
    print(f'  Links: {n_links}')
    print(f'  Loads: {n_loads}')
    
    return n

In [8]:
# Create detailed model
print('Creating detailed flexibility model...')
n_detailed = n_day.copy()

n_detailed = add_detailed_ev_flexibility(n_detailed, FLEX_PARAMS)
n_detailed = add_detailed_hp_flexibility(n_detailed, FLEX_PARAMS)

print(f'\nDetailed model summary:')
print(f'  Buses: {len(n_detailed.buses)}')
print(f'  Stores: {len(n_detailed.stores)}')
print(f'  Links: {len(n_detailed.links)}')
print(f'  Loads: {len(n_detailed.loads)}')

Creating detailed flexibility model...
Added detailed EV flexibility:
  Stores: 50
  Links: 50
  Loads: 50
Added detailed HP flexibility:
  Stores: 50
  Links: 50
  Loads: 50

Detailed model summary:
  Buses: 2608
  Stores: 101
  Links: 382
  Loads: 413


## 4. Model B: Simplified Load-Shifting Model

This model uses Generator/Load pairs to represent flexibility without explicit storage state tracking.

In [9]:
def add_simplified_flexibility(n, params):
    """
    Add simplified flexibility using load-shifting model.
    
    Components created per bus:
    - Generator (negative cost): Demand reduction capability
    - Load: Demand recovery (optional, for energy balance)
    
    This approach:
    - Has fewer variables (no SOC tracking)
    - Solves faster
    - Preserves key flexibility behaviors (peak shaving)
    - Trades off: less precise SOC constraints
    """
    load_buses = n.loads[n.loads.carrier != 'load_shedding'].bus.unique()
    
    # Add flexibility carrier
    if 'demand_response' not in n.carriers.index:
        n.add('Carrier', 'demand_response', co2_emissions=0.0)
    
    n_generators = 0
    
    # Calculate total flexible capacity
    ev_flex_fraction = params['ev_demand_fraction'] * params['ev_participation']
    hp_flex_fraction = params['hp_demand_fraction'] * params['hp_participation']
    total_flex_fraction = ev_flex_fraction + hp_flex_fraction
    
    for bus in load_buses[:50]:  # Limit for performance
        bus_loads = n.loads[n.loads.bus == bus]
        if bus_loads.empty:
            continue
            
        load_names = [l for l in bus_loads.index if l in n.loads_t.p_set.columns]
        if not load_names:
            continue
            
        bus_demand = n.loads_t.p_set[load_names].sum(axis=1)
        
        # Flexible capacity = fraction of demand that can be shifted
        flex_capacity_mw = bus_demand.max() * total_flex_fraction
        
        if flex_capacity_mw < 0.1:
            continue
        
        # Add demand response "generator" (represents demand reduction)
        # Low/negative marginal cost encourages use during high-price periods
        n.add('Generator',
              f"{bus} demand_response",
              bus=bus,
              carrier='demand_response',
              p_nom=flex_capacity_mw,
              marginal_cost=-10.0,  # Negative cost = incentive to use
              p_max_pu=1.0,
              p_min_pu=0.0)
        n_generators += 1
    
    print(f'Added simplified flexibility:')
    print(f'  DR Generators: {n_generators}')
    print(f'  Total flex fraction: {total_flex_fraction:.1%}')
    
    return n

In [10]:
# Create simplified model
print('Creating simplified flexibility model...')
n_simplified = n_day.copy()

n_simplified = add_simplified_flexibility(n_simplified, FLEX_PARAMS)

print(f'\nSimplified model summary:')
print(f'  Buses: {len(n_simplified.buses)}')
print(f'  Stores: {len(n_simplified.stores)}')
print(f'  Links: {len(n_simplified.links)}')
print(f'  Loads: {len(n_simplified.loads)}')
print(f'  Generators: {len(n_simplified.generators)}')

Creating simplified flexibility model...
Added simplified flexibility:
  DR Generators: 50
  Total flex fraction: 5.3%

Simplified model summary:
  Buses: 2508
  Stores: 1
  Links: 282
  Loads: 313
  Generators: 5035


## 5. Solve Both Models

In [11]:
# Solver configuration
SOLVER_NAME = 'highs'  # or 'gurobi' if available
SOLVER_OPTIONS = {'threads': 4}

# Try gurobi first, fall back to highs
try:
    import gurobipy
    SOLVER_NAME = 'gurobi'
    SOLVER_OPTIONS = {
        'threads': 4,
        'method': 2,
        'crossover': 0,
        'BarConvTol': 1e-4,
    }
    print('Using Gurobi solver')
except ImportError:
    print('Gurobi not available, using HiGHS solver')

Using Gurobi solver


In [12]:
# Solve detailed model
print('=' * 60)
print('SOLVING DETAILED MODEL')
print('=' * 60)

start_time = time.time()

status_detailed = n_detailed.optimize(
    solver_name=SOLVER_NAME,
    solver_options=SOLVER_OPTIONS
)

solve_time_detailed = time.time() - start_time

print(f'\nStatus: {status_detailed}')
print(f'Solve time: {solve_time_detailed:.1f} seconds')
if hasattr(n_detailed, 'objective'):
    print(f'Objective: £{n_detailed.objective:,.0f}')

SOLVING DETAILED MODEL


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - threads: 4
 - method: 2
 - crossover: 0
 - BarConvTol: 0.0001
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 9/9 [00:00<00:00, 11.48it/s]
INFO:linopy.io: Writing time: 7.75s


Set parameter Username


INFO:gurobipy:Set parameter Username


INFO:gurobipy:


...

INFO:gurobipy:Barrier solved model in 32 iterations and 35.53 seconds (17.50 work units)


Optimal objective 1.10925174e+06


INFO:gurobipy:Optimal objective 1.10925174e+06


INFO:gurobipy:
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 277680 primals, 662280 duals
Objective: 1.13e+06
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Transformer-fix-s-lower, Transformer-fix-s-upper, Link-fix-p-lower, Link-fix-p-upper, Store-fix-e-lower, Store-fix-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.



Status: ('ok', 'optimal')
Solve time: 68.3 seconds
Objective: £1,131,483


In [13]:
# Solve simplified model
print('=' * 60)
print('SOLVING SIMPLIFIED MODEL')
print('=' * 60)

start_time = time.time()

status_simplified = n_simplified.optimize(
    solver_name=SOLVER_NAME,
    solver_options=SOLVER_OPTIONS
)

solve_time_simplified = time.time() - start_time

print(f'\nStatus: {status_simplified}')
print(f'Solve time: {solve_time_simplified:.1f} seconds')
if hasattr(n_simplified, 'objective'):
    print(f'Objective: £{n_simplified.objective:,.0f}')

SOLVING SIMPLIFIED MODEL


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - threads: 4
 - method: 2
 - crossover: 0
 - BarConvTol: 0.0001
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 9/9 [00:00<00:00, 12.53it/s]
INFO:linopy.io: Writing time: 6.59s


Set parameter Username


INFO:gurobipy:Set parameter Username


INFO:gurobipy:


...

INFO:gurobipy:Barrier solved model in 38 iterations and 34.29 seconds (18.05 work units)


Optimal objective 9.90850972e+05


INFO:gurobipy:Optimal objective 9.90850972e+05


INFO:gurobipy:
Status: ok
Termination condition: suboptimal
Solution: 271680 primals, 650280 duals
Objective: 1.01e+06
Solver model: available
Solver message: 13

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Transformer-fix-s-lower, Transformer-fix-s-upper, Link-fix-p-lower, Link-fix-p-upper, Store-fix-e-lower, Store-fix-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.



Status: ('ok', 'suboptimal')
Solve time: 67.4 seconds
Objective: £1,006,185


## 6. Compare Results

In [14]:
# Summary comparison
print('=' * 80)
print('COMPARISON SUMMARY')
print('=' * 80)

comparison = {
    'Metric': [
        'Solve Time (s)',
        'Buses',
        'Stores',
        'Links',
        'Generators',
        'Status',
        'Objective (£)'
    ],
    'Detailed Model': [
        f'{solve_time_detailed:.1f}',
        len(n_detailed.buses),
        len(n_detailed.stores),
        len(n_detailed.links),
        len(n_detailed.generators),
        str(status_detailed),
        f'{n_detailed.objective:,.0f}' if hasattr(n_detailed, 'objective') else 'N/A'
    ],
    'Simplified Model': [
        f'{solve_time_simplified:.1f}',
        len(n_simplified.buses),
        len(n_simplified.stores),
        len(n_simplified.links),
        len(n_simplified.generators),
        str(status_simplified),
        f'{n_simplified.objective:,.0f}' if hasattr(n_simplified, 'objective') else 'N/A'
    ]
}

df_comparison = pd.DataFrame(comparison)
print(df_comparison.to_string(index=False))

# Speed improvement
if solve_time_detailed > 0:
    speedup = solve_time_detailed / solve_time_simplified
    print(f'\nSpeed improvement: {speedup:.1f}x faster')

COMPARISON SUMMARY
        Metric    Detailed Model     Simplified Model
Solve Time (s)              68.3                 67.4
         Buses              2608                 2508
        Stores               101                    1
         Links               382                  282
    Generators              4985                 5035
        Status ('ok', 'optimal') ('ok', 'suboptimal')
 Objective (£)         1,131,483            1,006,185

Speed improvement: 1.0x faster


In [15]:
# Compare generation mix
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['Detailed Model', 'Simplified Model'],
                    specs=[[{'type': 'pie'}, {'type': 'pie'}]])

for idx, (n, name) in enumerate([(n_detailed, 'Detailed'), (n_simplified, 'Simplified')], 1):
    if len(n.generators_t.p) > 0:
        gen_by_carrier = n.generators_t.p.sum().groupby(n.generators.carrier).sum()
        gen_by_carrier = gen_by_carrier[gen_by_carrier > 0]
        
        fig.add_trace(
            go.Pie(labels=gen_by_carrier.index, 
                   values=gen_by_carrier.values,
                   name=name),
            row=1, col=idx
        )

fig.update_layout(title='Generation Mix Comparison', height=400)
fig.show()

In [16]:
# Compare demand profiles
fig = go.Figure()

# Total demand (same for both models initially)
total_demand = n_day.loads_t.p_set.sum(axis=1)

fig.add_trace(go.Scatter(
    x=total_demand.index,
    y=total_demand / 1000,
    name='Original Demand',
    line=dict(color='blue', width=2)
))

# Net demand after flexibility (detailed)
if len(n_detailed.stores_t.p) > 0:
    store_power = n_detailed.stores_t.p.sum(axis=1)
    net_demand_detailed = total_demand - store_power
    fig.add_trace(go.Scatter(
        x=net_demand_detailed.index,
        y=net_demand_detailed / 1000,
        name='Net Demand (Detailed)',
        line=dict(color='red', width=2, dash='dash')
    ))

# DR dispatch (simplified)
dr_gens = n_simplified.generators[n_simplified.generators.carrier == 'demand_response']
if len(dr_gens) > 0 and len(n_simplified.generators_t.p) > 0:
    dr_cols = [g for g in dr_gens.index if g in n_simplified.generators_t.p.columns]
    if dr_cols:
        dr_dispatch = n_simplified.generators_t.p[dr_cols].sum(axis=1)
        net_demand_simplified = total_demand - dr_dispatch
        fig.add_trace(go.Scatter(
            x=net_demand_simplified.index,
            y=net_demand_simplified / 1000,
            name='Net Demand (Simplified)',
            line=dict(color='green', width=2, dash='dot')
        ))

fig.update_layout(
    title='Demand Profile Comparison',
    xaxis_title='Time',
    yaxis_title='Demand (GW)',
    height=500,
    template='plotly_white'
)
fig.show()

In [17]:
# Storage state of charge (detailed model only)
if len(n_detailed.stores_t.e) > 0:
    fig = go.Figure()
    
    # EV batteries
    ev_stores = n_detailed.stores[n_detailed.stores.carrier == 'EV battery']
    if len(ev_stores) > 0:
        ev_cols = [s for s in ev_stores.index if s in n_detailed.stores_t.e.columns]
        if ev_cols:
            ev_soc = n_detailed.stores_t.e[ev_cols].sum(axis=1)
            fig.add_trace(go.Scatter(
                x=ev_soc.index,
                y=ev_soc / 1000,
                name='EV Batteries',
                fill='tozeroy',
                line=dict(color='blue')
            ))
    
    # Hot water tanks
    hw_stores = n_detailed.stores[n_detailed.stores.carrier == 'hot water']
    if len(hw_stores) > 0:
        hw_cols = [s for s in hw_stores.index if s in n_detailed.stores_t.e.columns]
        if hw_cols:
            hw_soc = n_detailed.stores_t.e[hw_cols].sum(axis=1)
            fig.add_trace(go.Scatter(
                x=hw_soc.index,
                y=hw_soc / 1000,
                name='Hot Water Tanks',
                fill='tozeroy',
                line=dict(color='orange')
            ))
    
    fig.update_layout(
        title='Flexibility Storage State of Charge (Detailed Model)',
        xaxis_title='Time',
        yaxis_title='Energy (GWh)',
        height=400,
        template='plotly_white'
    )
    fig.show()
else:
    print('No store energy timeseries available')

## 7. Conclusions

In [18]:
print('=' * 80)
print('CONCLUSIONS')
print('=' * 80)

print(f'''
Detailed Model:
- Explicitly tracks storage state of charge (SOC)
- Models physical constraints (min SOC, standing losses, COP)
- Creates {len(n_detailed.stores)} Store and {len(n_detailed.links)} Link components
- Solve time: {solve_time_detailed:.1f} seconds

Simplified Model:
- Models flexibility as load-shifting generators
- No explicit SOC tracking
- Creates {len(n_simplified.generators) - len(n_day.generators)} additional Generator components
- Solve time: {solve_time_simplified:.1f} seconds

Trade-offs:
+ Simplified model is {solve_time_detailed/solve_time_simplified:.1f}x faster
+ Simplified model has fewer variables and constraints
- Simplified model doesn't track exact SOC evolution
- Simplified model may over/underestimate flexibility in some cases

Recommendation:
- For rapid prototyping and year-long simulations: Use Simplified Model
- For detailed analysis of flexibility behavior: Use Detailed Model
- For production runs with many scenarios: Consider hybrid approach
''')

CONCLUSIONS

Detailed Model:
- Explicitly tracks storage state of charge (SOC)
- Models physical constraints (min SOC, standing losses, COP)
- Creates 101 Store and 382 Link components
- Solve time: 68.3 seconds

Simplified Model:
- Models flexibility as load-shifting generators
- No explicit SOC tracking
- Creates 50 additional Generator components
- Solve time: 67.4 seconds

Trade-offs:
+ Simplified model is 1.0x faster
+ Simplified model has fewer variables and constraints
- Simplified model doesn't track exact SOC evolution
- Simplified model may over/underestimate flexibility in some cases

Recommendation:
- For rapid prototyping and year-long simulations: Use Simplified Model
- For detailed analysis of flexibility behavior: Use Detailed Model
- For production runs with many scenarios: Consider hybrid approach



In [19]:
# Save results for future reference
results_dir = project_root / 'resources' / 'analysis'
results_dir.mkdir(parents=True, exist_ok=True)

df_comparison.to_csv(results_dir / 'demand_flexibility_comparison.csv', index=False)
print(f'Results saved to {results_dir / "demand_flexibility_comparison.csv"}')

Results saved to c:\Users\alyden\OneDrive - University of Edinburgh\Python\PyPSA-GB v0.0.1\resources\analysis\demand_flexibility_comparison.csv
